In [14]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR
import random
from scipy import sparse as sps
import importlib

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
train, test, tgt_tracks, tgt_playlists = rs.split_train_test(train, 10, 20, 5, 2517)
tgt_playlists.shape

(4420, 1)

In [4]:
ibr = IBR.ItemBasedRecommender()
ibr.fit(tracks,train,tgt_tracks)
#fit(self, track_ids, train_data, tgt_tracks=None):
print('Model fitted!')

  1%|          | 33/4420 [00:00<00:13, 321.79it/s]

Calculated Indices


  0%|          | 36/100000 [00:00<04:41, 355.60it/s]

(4420, 100000)
Model URM built


100%|██████████| 100000/100000 [07:53<00:00, 211.38it/s]

Similarity built
(100000, 17113)
Model fitted!


In [8]:
tsr= TSR.TopSimilarRecommender(attributes=['artist_id', 'album', 'tags', 'playcount'],attributes_to_prune= ['tags'],idf=True, n_min_attr=90)
tsr.fit(tracks,tgt_tracks )
print('Model fitted!')

  7%|▋         | 6972/100000 [00:30<06:49, 227.14it/s]

Fixed dataset
Calculated Indices
Eliminated low frequency attributes!
ICM built
ICM regularized with IDF!



  1%|          | 1203/100000 [00:04<06:43, 244.60it/s]
Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/giada/anaconda3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/giada/anaconda3/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/giada/anaconda3/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 100000/100000 [09:13<00:00, 180.83it/s]

Similarity built
Model fitted!


In [81]:
rs = importlib.reload(rs)
similarity_matrix = rs. merge_similarities(tsr.S, ibr.S, 0.2)

In [82]:
similarity_matrix

<100000x17113 sparse matrix of type '<class 'numpy.float64'>'
	with 2965233 stored elements in Compressed Sparse Row format>

# Recommendation

In [83]:
_, _, IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists)
URM = rs.create_tgt_URM(IX_tgt_playlists, tsr.IX_items, train)
URM = URM.tocsr()
print('URM built')

100%|██████████| 4420/4420 [00:08<00:00, 495.57it/s]


URM built


In [84]:
rs = importlib.reload(rs)
recommendetions = np.array([])

for p in IX_tgt_playlists.values:
    #if normalize:
    #    tsr_avg_sims = np.divide(URM[p,:].dot(TopSimilarRecommender.S).toarray(), norm_dividend).ravel()
    avg_sims = URM[p,:].dot(similarity_matrix).toarray().ravel()
    top = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))

rec_df =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

Recommended 0 users over 4420
Recommended 1000 users over 4420
Recommended 2000 users over 4420
Recommended 3000 users over 4420
Recommended 4000 users over 4420


# Evaluation

In [ ]:
def evaluate(results, test, eval_metric='MAP'):
    if eval_metric == 'MAP':
        APs = results.apply(calculate_AP, axis=1, args=(test,))
        res = (APs.sum())/results.shape[0]
    return res

def calculate_AP(row, test):
    p_id = row['playlist_id']
    recs = np.fromstring(row['track_ids'], dtype=float, sep=' ')

    AP = 0
    rel_sum = 0
    n_rel_items = min(test[test['playlist_id'] == p_id].shape[0],5)
    for i in range(recs.size):
        rel = 1 if ((test['playlist_id'] == int(p_id)) & (test['track_id'] == recs[i])).any() else 0
        rel_sum += rel
        P = rel_sum/(i+1)
        AP += (P * rel)/n_rel_items

    return AP

In [ ]:
map_eval = evaluate(rec_df, test, 'MAP')
print('Evaluation completed!')

In [ ]:
map_eval